In [9]:
import pandas as pd
import warnings
warnings.filterwarnings(action = 'ignore')

final_df = pd.read_csv('/content/drive/MyDrive/VOD Project/준호/데이터스쿨3차_2308월/final_df.csv', index_col=0)
final_df

,userid,program,score,main_cat,sub_cat
0,59879000,소방서 옆 경찰서,0.122238,TV드라마,기타
1,59879000,신성한 이혼,0.861328,TV드라마,기타
2,59900000,초대: 스와핑 데이,0.292893,영화,멜로
3,59900000,후궁-제왕의첩,0.292893,영화,멜로
4,59900000,범죄도시3,0.250000,영화,액션/어드벤쳐
...,...,...,...,...,...
1257,67107000,고창 2부,0.000000,우리동네,연예/오락
1258,67107000,마크맨,0.292893,영화,액션/어드벤쳐
1259,67107000,고창 1부,0.000000,우리동네,연예/오락
1260,67107000,전라남도 여수 2부,0.000000,우리동네,연예/오락


In [10]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# 사용자-항목 행렬 생성을 위해 pivot_table 사용
user_program_matrix = final_df.pivot_table(index='userid', columns='program', values='score').fillna(0)

# 사용자 기반 협업 필터링을 위해 NearestNeighbors 모델 생성
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

# 모델 학습
model_knn.fit(user_program_matrix)

# 중복을 허용하여 추천 프로그램 목록을 생성하는 함수 수정
def recommend_programs_with_duplicates(user_id, user_program_matrix, model_knn, n_recommendations=5):
    if user_id not in user_program_matrix.index:
        return "User ID not found in dataset."

    # 입력된 사용자 ID에 대한 데이터 추출
    user_data = user_program_matrix.loc[user_id].values.reshape(1, -1)

    # 가장 유사한 사용자들 찾기
    distances, indices = model_knn.kneighbors(user_data, n_neighbors=n_recommendations+1)

    recommendations = []
    for i in range(1, len(distances.flatten())):
        similar_user_id = user_program_matrix.index[indices.flatten()[i]]
        similar_user_data = user_program_matrix.loc[similar_user_id]
        recommended_programs = similar_user_data[similar_user_data > 0].index.tolist()
        recommendations.extend(recommended_programs)

    # 중복을 허용하되, 추천 수를 제한하여 최종 추천 목록 반환
    return recommendations[:n_recommendations]

# 테스트: 임의의 사용자 ID에 대한 추천 프로그램 목록 출력
user_id = final_df['userid'].unique()
test_user_id = user_id[0]
recommend_programs_with_duplicates(test_user_id, user_program_matrix, model_knn, n_recommendations = 10)

['NFS 국과수',
 '감각의 제국 감독판',
 '과학수사대 스모킹 건',
 '국민사형투표',
 '동상이몽 2-너는 내 운명',
 '소방서 옆 경찰서',
 '소방서 옆 경찰서 그리고 국과수',
 '연애담',
 '이슈 픽 쌤과 함께',
 '전국 노래 자랑']

In [ ]:
# !pip install surprise

In [17]:
# suprise 라이브러리 사용
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
from collections import defaultdict

# 데이터셋을 surprise 라이브러리가 사용할 수 있는 형태로 변환
reader = Reader(rating_scale=(0, 5))  # 평점 스케일 지정
data = Dataset.load_from_df(final_df[['userid', 'program', 'score']], reader)

# 데이터를 훈련 세트와 테스트 세트로 분할
trainset, testset = train_test_split(data, test_size=0.25)

# KNNBasic 알고리즘을 사용
algo = KNNBasic()

# 훈련 세트로 모델 학습
algo.fit(trainset)

# 이미 시청한 항목 포함 추천 함수
def get_recommendations_including_watched(test_user_id, trainset, algo, n=10):
    watched_items = [i for (u, i, r) in trainset.all_ratings() if u == trainset.to_inner_uid(test_user_id)]
    all_items = set(trainset.all_items())
    not_watched_items = all_items - set(watched_items)

    # 이미 시청한 항목에 대한 예측 평점 추가
    predictions = [algo.predict(test_user_id, trainset.to_raw_iid(i)) for i in watched_items]
    # 아직 시청하지 않은 항목에 대한 예측 평점 추가
    predictions += [algo.predict(test_user_id, trainset.to_raw_iid(i)) for i in not_watched_items]

    # 예측 평점 기준으로 정렬
    predictions.sort(key=lambda x: x.est, reverse=True)

    # 상위 N개의 추천 항목 반환
    return [pred.iid for pred in predictions[:n]]

# 특정 사용자에 대한 추천 항목 출력 (이미 시청한 항목 포함)
user_id = final_df['userid'].unique()
test_user_id = user_id[0]
recommended_items = get_recommendations_including_watched(test_user_id, trainset, algo, n=10)
print(f"userID : {test_user_id} \nRecommend :\n{recommended_items}")

Computing the msd similarity matrix...
Done computing similarity matrix.
userID : 59879000 
Recommend :
['신성한 이혼', '낭만닥터 김사부', '소방서 옆 경찰서 그리고 국과수', '천원짜리 변호사', '쾌걸 흑두건', ' 고창 1부', '서프라이즈 장난감 놀이', '러브 포 세일', '스트릿 우먼 파이터', '몬스터 신부: 101번째 프로포즈']


In [23]:
# 하이브리드 필터링
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

# 콘텐츠 기반 필터링을 위한 TF-IDF 벡터 생성
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(final_df['sub_cat'])  # 장르 컬럼 사용
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# 협업 필터링을 위한 사용자-프로그램 행렬 생성
user_series_matrix = final_df.pivot_table(index='userid', columns='program', values='score').fillna(0)
user_series_sparse = csr_matrix(user_series_matrix.values)

# 모델 생성 및 학습
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
model_knn.fit(user_series_sparse)

def hybrid_recommendation(user_id, series_name):
    # 콘텐츠 기반 추천
    idx = final_df[final_df['program'] == series_name].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    series_indices = [i[0] for i in sim_scores]

    # 협업 필터링 추천
    distances, indices = model_knn.kneighbors(user_series_matrix.loc[user_id].values.reshape(1, -1))
    similar_users = [user_series_matrix.index[i] for i in indices.flatten()[1:]]
    recommended_series = set()
    for user in similar_users:
        recommended_series.update(user_series_matrix.loc[user].index[user_series_matrix.loc[user].values > 0])

    # 두 방법의 결과 결합
    hybrid_recommendations = []
    for i in series_indices:
        series_name = final_df['program'][i]
        if series_name in recommended_series and series_name not in hybrid_recommendations:
            hybrid_recommendations.append(series_name)
        if len(hybrid_recommendations) == 10:
            break

    return hybrid_recommendations

# 테스트: 임의의 사용자 ID와 프로그램 시리즈명에 대한 하이브리드 추천 목록 출력
test_user_id = final_df['userid'].iloc[0]
test_series_name = final_df['program'].iloc[0]
hybrid_recommendation(test_user_id, test_series_name)

['소방서 옆 경찰서',
 '런닝맨',
 '진짜가 나타났다!',
 '연인 파트1',
 '힙하게',
 '소방서 옆 경찰서 그리고 국과수',
 '심야괴담회',
 '동상이몽 2-너는 내 운명',
 '꼬리에꼬리를무는그날이야기',
 '고딩엄빠 3']